# Lab15

## Consultas requeridas
1. Obtener todas las mediciones de humedad del sensor 'SENS001' del último día. La consulta debe incluir el tiempo restante de vida (TTL) de cada registro.

2. Detectar valores anómalos fuera del rango permitido en la última hora. Implementar una consulta que identifique mediciones de temperatura o humedad fuera del rango normal. La consulta debe permitir filtrar por sensor específico.

3. Verificar el tiempo restante de vida de los datos usando la función TTL. Implementar una consulta que muestre el TTL en diferentes unidades (segundos, horas, días). Crear una consulta para identificar datos que están próximos a expirar (ej: en las próximas 24 horas).

## Estructura de tabla propuesta
La tabla *sensor_readings* tendrá un propósito general que permite filtrar por tipo de medicion, sensor y día. Esta tabla se usará para la consulta 1.

La tabla *sensor_anomalies* será destinada a optimizar la consulta 2, esta nos permite filtrar por hora ya que se incluye este dato en el partition key, además de tener clustering por el valor de la medición lo que facilita hallar los valores anómalos.

La tabla *sensor_by_date* fue pensada para usarse con la consulta 3, porque particiona solo por la fecha y con esto poder filtrar los datos con más de 6 días de antigüedad.

Se respeta el tiempo de vida de los datos de 7 días de acuerdo a lo solicitado, y se considera adicionalmente un tiempo de vida de solo 2 horas para el seguimiento de datos anómalos ya que se espera que siempre se consulten los insertados en la hora pasada.

In [ ]:
create table if not exists sensor_readings
(
    measurement_type text,
    sensor_id        text,
    date             text,
    event_time       timestamp,
    measurement      double,
    primary key ( (measurement_type, sensor_id, date), event_time )
) with clustering order by (event_time desc) and
        default_time_to_live = 604800; -- 7 days

create table if not exists sensor_anomalies
(
    measurement_type text,
    sensor_id        text,
    hour             text,
    event_time       timestamp,
    measurement      double,
    primary key ( (measurement_type, sensor_id, hour), measurement, event_time)
) with clustering order by (measurement asc, event_time desc) and
        default_time_to_live = 7200; -- 2 hour

create table if not exists sensor_by_date
(
    measurement_type text,
    sensor_id        text,
    date             text,
    event_time       timestamp,
    measurement      double,
    primary key ( date, event_time )
) with clustering order by (event_time desc) and
        default_time_to_live = 604800; -- 7 days

## Creación de datos de prueba

In [4]:
%pip install cassandra-driver

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import random
import time
from datetime import datetime, timedelta
from cassandra.cluster import Cluster

# Parametros de simulación
N_SENSORS = 5
SAMPLING_RATE = 60 # 1 minuto en segundos
SAMPLING_TIME = 604800  # 7 días en segundos
ID_PREFIX = 'SENS'
MESUREMENTS_TYPES = ('temperatura', 'humedad')
NORMAL_RANGE = {'temperatura': (15, 35),
                'humedad': (30, 80)}

# Precomputar rango extendido de valores para simulación de anomalías
EXTENDED_RANGE = {
    type: (low - (high - low) * 0.2,
        high + (high - low) * 0.2
    )
    for type, (low, high) in NORMAL_RANGE.items()
}

# Conexión a Cassandra en Docker
cluster = Cluster(['localhost'], port=9042)
cassandra = cluster.connect('my_keyspace')

def create_tables() -> None:
    # Crear tabla para lecturas de sensores
    cassandra.execute("""
        create table if not exists sensor_readings
        (
            measurement_type text,
            sensor_id        text,
            date             text,
            event_time       timestamp,
            measurement      double,
            primary key ( (measurement_type, sensor_id, date), event_time )
        ) with clustering order by (event_time desc) and
                default_time_to_live = 604800
    """)
    cassandra.execute("""
        create table if not exists sensor_anomalies
        (
            measurement_type text,
            sensor_id        text,
            hour             text,
            event_time       timestamp,
            measurement      double,
            primary key ( (measurement_type, sensor_id, hour), measurement, event_time)
        ) with clustering order by (measurement asc, event_time desc) and
                default_time_to_live = 7200
    """)
    cassandra.execute("""
        create table if not exists sensor_by_date
        (
            measurement_type text,
            sensor_id        text,
            date             text,
            event_time       timestamp,
            measurement      double,
            primary key ( date, event_time )
        ) with clustering order by (event_time desc) and
                default_time_to_live = 604800
    """)

def drop_tables() -> None:
    # Eliminar tablas si existen
    cassandra.execute("drop table if exists sensor_readings")
    cassandra.execute("drop table if exists sensor_anomalies")
    cassandra.execute("drop table if exists sensor_by_date")
    
def generate_sensor_data() -> None:
    # Generar indetificadores únicos para cada sensor
    sensor_id: list[str] = [f"{ID_PREFIX}{str(i).zfill(3)}" for i in range(1, N_SENSORS + 1)]

    # Definir el tiempo de inicio y fin para la generación de datos
    start_time: datetime = datetime.now() - timedelta(seconds=SAMPLING_TIME)
    end_time: datetime = datetime.now()
    current_time: datetime = start_time

    while current_time <= end_time:
        # Extraer fecha y hora del timestamp actual
        date: str = current_time.strftime('%Y-%m-%d')
        hour: str = current_time.strftime('%Y-%m-%dT%H')

        for id in sensor_id:
            for type in MESUREMENTS_TYPES:
                ext_low, ext_high = EXTENDED_RANGE[type]
                # Generar un valor en el rango extendido
                measurement: float = random.uniform(ext_low, ext_high)
                # Insertar en tablas
                cassandra.execute("""
                    insert into sensor_readings(
                        measurement_type, sensor_id, date, event_time, measurement
                    ) values (%s, %s, %s, %s, %s)
                """, (type, id, date, current_time, measurement))
                cassandra.execute("""
                    insert into sensor_anomalies(
                        measurement_type, sensor_id, hour, event_time, measurement
                    ) values (%s, %s, %s, %s, %s)
                """, (type, id, hour, current_time, measurement))
                cassandra.execute("""
                    insert into sensor_by_date(
                        measurement_type, sensor_id, date, event_time, measurement
                    ) values (%s, %s, %s, %s, %s)
                """, (type, id, date, current_time, measurement))
        # Avanzar al siguiente intervalo de muestreo
        current_time += timedelta(seconds=SAMPLING_RATE)
    
    print("Datos de prueba generados exitosamente.")

def query_1(type: str, id: str) -> None:
    print("\nQuery 1:")
    # Calcular el bucket del día anterior
    target_date: str = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    # Ejecutar consulta y medir el tiempo de ejecución
    start_time = time.time()
    rows = cassandra.execute("""
        select event_time, measurement, ttl(measurement) as ttl
        from sensor_readings
        where measurement_type = %s
            and sensor_id = %s
            and date = %s
    """, (type, id, target_date))
    end_time = time.time()
    query_time = end_time - start_time
    rows: list = list(rows)  # Convertir a lista para poder usar len() y slicing
    print(f"Lecturas de {type} para el sensor {id} del día de ayer {target_date}:")
    print(f"{len(rows)} resultados encontrados en {query_time:.4f} segundos")
    for row in rows[:5]:  # Limitar a las primeras 5 lecturas
        event_time = row.event_time.strftime('%Y-%m-%d %H:%M:%S')
        measurement = row.measurement
        ttl_seconds = row.ttl
        print(f"  - {event_time}: {measurement} (TTL: {ttl_seconds} segundos)")

def query_2(type: str, id: str) -> None:
    print("\nQuery 2:")
    # Calcular el bucket de la hora anterior
    target_hour: str = (datetime.now() - timedelta(hours=1)).strftime('%Y-%m-%dT%H')
    # Obtener el rango normal para el tipo de medición
    low, high = NORMAL_RANGE[type]
    # Ejecutar consulta y medir el tiempo de ejecución
    start_time = time.time()
    rows_low = cassandra.execute("""
        select event_time, measurement
        from sensor_anomalies
        where measurement_type = %s
            and sensor_id = %s
            and hour = %s
            and measurement < %s
    """, (type, id, target_hour, low))

    rows_high = cassandra.execute("""
        select event_time, measurement
        from sensor_anomalies
        where measurement_type = %s
            and sensor_id = %s
            and hour = %s
            and measurement > %s
    """, (type, id, target_hour, high))
    end_time = time.time()
    query_time = end_time - start_time
    rows: list = list(rows_low) + list(rows_high)
    rows.sort(key=lambda row: row.event_time, reverse=True)
    print(f"Anomalías de {type} para el sensor {id} de la hora pasada {target_hour}:")
    print(f"{len(rows)} resultados encontrados en {query_time:.4f} segundos")
    for row in rows[:5]:  # Limitar a las primeras 5 anomalías
        event_time = row.event_time.strftime('%Y-%m-%d %H:%M:%S')
        measurement = row.measurement
        print(f"  - {event_time}: {measurement}")

def query_3() -> None:
    print("\nQuery 3:")
    # Calcular el bucket de la fecha de hace 6 días
    target_date: str = (datetime.now() - timedelta(days=6)).strftime('%Y-%m-%d')
    # Ejecutar consulta y medir el tiempo de ejecución
    start_time = time.time()
    rows = cassandra.execute("""
        select measurement_type, sensor_id, date, event_time, ttl(measurement) as ttl
        from sensor_by_date
        where date = %s
    """, (target_date,))
    end_time = time.time()
    query_time = end_time - start_time
    rows: list = list(rows)  # Convertir a lista para poder usar len() y slicing
    print(f"Datos de más de 6 días de antigüedad ({target_date}):")
    print(f"{len(rows)} resultados encontrados en {query_time:.4f} segundos")
    for row in rows[:5]:  # Limitar a las primeras 5 lecturas
        measurement_type = row.measurement_type
        sensor_id = row.sensor_id
        date = row.date
        ttl_seconds = row.ttl
        ttl_minutes = ttl_seconds // 60
        ttl_hours = ttl_minutes // 60
        ttl_days = ttl_hours // 24
        print(f"  - {measurement_type} del sensor {sensor_id} del día {date} (TTL: {ttl_seconds} segundos, {ttl_minutes} minutos, {ttl_hours} horas, {ttl_days} días)")

def test() -> None:
    # Crear tablas
    create_tables()
    
    # Generar datos de prueba
    generate_sensor_data()
    
    # Ejecutar consultas de prueba
    query_1('temperatura', 'SENS001')
    query_2('humedad', 'SENS002')
    query_3()
    
    # Limpiar tablas
    drop_tables()

test()
# Cerrar conexión a Cassandra
cluster.shutdown()

Datos de prueba generados exitosamente.

Query 1:
Lecturas de temperatura para el sensor SENS001 del día de ayer 2025-07-07:
12 resultados encontrados en 0.0163 segundos
  - 2025-07-07 22:22:09: 27.575427960880024 (TTL: 604799 segundos)
  - 2025-07-07 20:22:09: 34.7803031711525 (TTL: 604798 segundos)
  - 2025-07-07 18:22:09: 34.472052054034165 (TTL: 604798 segundos)
  - 2025-07-07 16:22:09: 22.855188008731396 (TTL: 604797 segundos)
  - 2025-07-07 14:22:09: 15.096105327635264 (TTL: 604797 segundos)

Query 2:
Anomalías de humedad para el sensor SENS002 de la hora pasada 2025-07-08T01:
0 resultados encontrados en 0.0280 segundos

Query 3:
Datos de más de 6 días de antigüedad (2025-07-02):
12 resultados encontrados en 0.0159 segundos
  - humedad del sensor SENS005 del día 2025-07-02 (TTL: 604771 segundos, 10079 minutos, 167 horas, 6 días)
  - humedad del sensor SENS005 del día 2025-07-02 (TTL: 604771 segundos, 10079 minutos, 167 horas, 6 días)
  - humedad del sensor SENS005 del día 2025-07